In [1]:


import sys
from concurrent import futures
import numpy as np
from pyspark.sql import SparkSession
from datetime import datetime,timedelta
import pytz

# print(datetime.now(pytz.timezone("Asia/Shanghai")).strftime( '%Y-%m-%d'))
today=(datetime.now()+ timedelta(-1)).strftime( '%Y-%m-%d')
yesterday=(datetime.now(pytz.timezone("Asia/Shanghai"))+ timedelta(-1)).strftime( '%Y-%m-%d')
# tablemap={"telemarket_stream_etl":"telemarket"} 
tablemap={"telemarket_stream_etl":"telemarket"} 
# ,"banda_stream_etl":"1"
tablesql_1=" FROM  (   SELECT    *  , row_number() OVER (PARTITION BY id  ORDER BY date_format(etldate,'yyyy-MM-dd HH:mm:ss.SSS') DESC  , if(etlindex is null ,0,etlindex)  desc)  row_num FROM  "
tablesql_2=" ORDER BY id  ASC )  a  WHERE a.row_num = 1 AND a.kind <> 'delete' "
# def getTableColum(b):
#     colum="";
#     for index in range(len(b)):
#         if(index>0 and index<len(b)-8):   
#             colum=colum+b[index]["col_name"]+", "
#     return colum+"year,month,day"
def getTableColum(b):
    colum=""
    for index in range(len(b)):
        if(index>0 and index<len(b)-7):   
            colum=colum+setDef(b[index]["data_type"],b[index]["col_name"])+", "
    return colum[0:len(colum)-2]
def setDef(type,table_col):
    if(type[:7] == 'decimal'):
        return "ifnull("+table_col+",0) as "+table_col
    else:
        return table_col
def make_col_temptable(databaseName,row):
    tableName=row["tableName"]
#     if tableName=='t_admin':
    tablecolum="desc " +databaseName+"."+tableName;
    tableSchema= spark.sql(tablecolum).collect()
    tablepath="s3://rupiahplus-data-warehouse/aliyun/"+tablemap[databaseName]+"/"+row["tableName"]
    spark.catalog.refreshTable(databaseName+"."+tableName)
    sql="select "+ getTableColum(tableSchema)+ tablesql_1+databaseName+"."+tableName+tablesql_2
#     print(sql)
    spark.sql(sql).drop("etlindex").write.mode("overwrite").partitionBy("year").orc(tablepath)
# __main__
#     
if __name__ == "builtins":
    spark = SparkSession\
        .builder\
        .appName("Python Demo")\
        .config("hive.metastore.client.factory.class", "com.amazonaws.glue.catalog.metastore.AWSGlueDataCatalogHiveClientFactory") \
        .enableHiveSupport()\
        .getOrCreate()
#     spark.conf.set("spark.scheduler.mode","FAIR")
    spark.conf.set("spark.sql.shuffle.partitions",20)
    for databaseName in tablemap:
        print(databaseName)
        databasesql="show tables in "+databaseName
        tables=spark.sql(databasesql)
        tablelist=tables.collect();
        executor=None
        with futures.ThreadPoolExecutor(max_workers=10) as executor:
            futures_result=futures.wait([executor.submit(make_col_temptable,databaseName, tableNm) for tableNm in tablelist])
            for  future in futures_result[0]:
                print(future.result())


VBox()

Starting Spark application


KeyboardInterrupt: 